In [134]:
from datasets import load_dataset

# On charge directement le split train/validation/test
dataset = load_dataset("ibm-research/argument_quality_ranking_30k", "argument_quality_ranking")

# Aperçu rapide
print(dataset)
dataset["train"][0]

DatasetDict({
    train: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 20974
    })
    validation: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 3208
    })
    test: Dataset({
        features: ['argument', 'topic', 'set', 'WA', 'MACE-P', 'stance_WA', 'stance_WA_conf'],
        num_rows: 6315
    })
})


{'argument': '"marriage" isn\'t keeping up with the times.  abandon the old thinking and bring something that incorporates all unions - not just those with a man and woman.',
 'topic': 'We should abandon marriage',
 'set': 'train',
 'WA': 0.846164614,
 'MACE-P': 0.297659077,
 'stance_WA': 1,
 'stance_WA_conf': 1.0}

In [135]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

# Fonction de tokenisation
def preprocess(examples):
    return tokenizer(
        examples["argument"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

# Appliquez la tokenisation sans retirer toutes les colonnes
tokenized = dataset.map(
    preprocess,
    batched=True,
    # ne retirez QUE les colonnes texte brutes inutiles,
    # mais conservez MACE-P
    remove_columns=["argument", "topic", "set", "WA", "stance_WA", "stance_WA_conf"]
)

In [136]:
import tensorflow as tf

def to_tf_dataset(split):
    # Features: input_ids, attention_mask → X ; label: MACE-P → y
    features = {
        "input_ids": tf.constant(split["input_ids"], dtype=tf.int32),
        "attention_mask": tf.constant(split["attention_mask"], dtype=tf.int32),
    }
    labels = tf.constant(split["MACE-P"], dtype=tf.float32)
    return tf.data.Dataset.from_tensor_slices((features, labels))

train_ds = to_tf_dataset(tokenized["train"]).shuffle(10_000).batch(32)
val_ds   = to_tf_dataset(tokenized["validation"]).batch(32)
test_ds  = to_tf_dataset(tokenized["test"]).batch(32)

In [137]:
import tensorflow as tf
from tensorflow.keras.optimizers.schedules import LearningRateSchedule, CosineDecayRestarts

# 1. CosineDecayRestarts pour snapshots et redémarrages
decay_schedule = CosineDecayRestarts(
    initial_learning_rate=3e-5,
    first_decay_steps=1000,   # ajuster selon nb_steps
    t_mul=2.0,
    m_mul=1.0,
    alpha=0.0
)

# 2. Classe WarmUp pour une montée linéaire des LR
class WarmUp(LearningRateSchedule):
    def __init__(self, initial_lr, decay_schedule_fn, warmup_steps):
        super().__init__()
        # Conservez les valeurs pour la configuration
        self.initial_lr = float(initial_lr)
        self.warmup_steps = int(warmup_steps)
        self.decay_fn = decay_schedule_fn

    def __call__(self, step):
        step_float = tf.cast(step, tf.float32)
        warmup_lr = tf.cast(self.initial_lr, tf.float32) * (step_float / tf.cast(self.warmup_steps, tf.float32))
        decay_lr = self.decay_fn(step - self.warmup_steps)
        return tf.where(step_float < self.warmup_steps, warmup_lr, decay_lr)

    def get_config(self):
        # Nécessaire pour la sérialisation du scheduler
        return {
            "initial_lr": self.initial_lr,
            "warmup_steps": self.warmup_steps,
            "decay_schedule_fn": tf.keras.optimizers.schedules.deserialize(
                {
                    "class_name": self.decay_fn.__class__.__name__,
                    "config": self.decay_fn.get_config()
                }
            )
        }

lr_schedule = WarmUp(
    initial_lr=3e-5,
    decay_schedule_fn=decay_schedule,
    warmup_steps=500
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [138]:
from tensorflow.keras import layers

class PositionalEncoding(layers.Layer):
    def __init__(self, max_len, d_model):
        super().__init__()
        pos = tf.range(max_len, dtype=tf.float32)[:, tf.newaxis]
        i   = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]
        angle = pos / tf.pow(10000.0, (2 * (i//2)) / tf.cast(d_model, tf.float32))
        pe = tf.where(tf.cast(i, tf.int32) % 2 == 0, tf.sin(angle), tf.cos(angle))
        self.pe = pe[tf.newaxis, ...]

    def call(self, x):
        return x + self.pe[:, :tf.shape(x)[1], :]

def build_transformer_model(
    vocab_size: int,
    max_len: int = 128,
    d_model: int = 128,
    num_heads: int = 4,
    ff_dim: int = 256,
    num_layers: int = 2,
):
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    att_mask  = layers.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")

    x = layers.Embedding(vocab_size, d_model)(input_ids)
    x = layers.Dropout(0.1)(x)                       # dropout sur embeddings

    x = PositionalEncoding(max_len, d_model)(x)

    for _ in range(num_layers):
        # Multi-head Self-Attention
        attn_output = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=d_model,
            dropout=0.1
        )(x, x, attention_mask=att_mask[:, tf.newaxis, tf.newaxis, :])
        attn_output = layers.Dropout(0.1)(attn_output)  # dropout sur attention
        attn_output = layers.LayerNormalization(epsilon=1e-6)(x + attn_output)

        # Feed-forward
        ff = layers.Dense(ff_dim, activation="relu")(attn_output)
        ff = layers.Dropout(0.1)(ff)                   # dropout feed-forward
        ff = layers.Dense(d_model)(ff)
        x = layers.LayerNormalization(epsilon=1e-6)(attn_output + ff)

    x = layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(1, activation="linear", name="mace_p")(x)

    return tf.keras.Model(inputs=[input_ids, att_mask], outputs=output)

vocab_size = tokenizer.vocab_size  # supposant que tokenizer est déjà chargé
model = build_transformer_model(vocab_size)
model.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_ids           │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 128, 128)  │  3,906,816 │ input_ids[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_23          │ (None, 128, 128)  │          0 │ embedding_4[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mask      │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ positional_encodin… │ (None, 128, 128)  │          0 │ dropout_23[0][0]  │
│ (PositionalEncodin… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_8          │ (None, 1, 1, 128) │          0 │ attention_mask[0… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 128)  │    263,808 │ positional_encod… │
│ (MultiHeadAttentio… │                   │            │ positional_encod… │
│                     │                   │            │ get_item_8[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_25          │ (None, 128, 128)  │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_16 (Add)        │ (None, 128, 128)  │          0 │ positional_encod… │
│                     │                   │            │ dropout_25[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 128)  │        256 │ add_16[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_16 (Dense)    │ (None, 128, 256)  │     33,024 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_26          │ (None, 128, 256)  │          0 │ dense_16[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_17 (Dense)    │ (None, 128, 128)  │     32,896 │ dropout_26[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_17 (Add)        │ (None, 128, 128)  │          0 │ layer_normalizat… │
│                     │                   │            │ dense_17[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 128, 128)  │        256 │ add_17[0][0]      │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_9          │ (None, 1, 1, 128) │          0 │ attention_mask[0… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 128, 128)  │    263,808 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat

 Total params: 4,567,425 (17.42 MB)

 Trainable params: 4,567,425 (17.42 MB)

 Non-trainable params: 0 (0.00 B)

In [139]:
from tensorflow.keras.callbacks import Callback

class UnfreezeCallback(Callback):
    def __init__(self, freeze_epochs=2):
        super().__init__()
        self.freeze_epochs = freeze_epochs

    def on_train_begin(self, logs=None):
        # geler embeddings et première couche d'attention
        for layer in self.model.layers:
            if isinstance(layer, tf.keras.layers.Embedding) or 'multi_head_attention' in layer.name:
                layer.trainable = False
        print("Couches initiales gelées")

    def on_epoch_end(self, epoch, logs=None):
        if epoch + 1 == self.freeze_epochs:
            for layer in self.model.layers:
                layer.trainable = True
            print(f"Couches dé-gelées à l'époque {epoch+1}")

unfreeze_cb = UnfreezeCallback(freeze_epochs=2)

In [140]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

model.compile(
    optimizer=optimizer,                            # scheduler + Adam
    loss="mean_squared_error",
    metrics=[tf.keras.metrics.MeanAbsoluteError(name="MAE")]
)

checkpoint_cb = ModelCheckpoint(
    "best_model.h5",
    monitor="val_MAE",
    mode="min",
    save_best_only=True,
    verbose=1
)
earlystop_cb = EarlyStopping(
    monitor="val_MAE",
    patience=3,
    mode="min",
    restore_best_weights=True,
    verbose=1
)

callbacks = [checkpoint_cb, earlystop_cb, unfreeze_cb]

In [141]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks
)

Couches initiales gelées
Epoch 1/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - MAE: 0.3383 - loss: 0.1542
Epoch 1: val_MAE improved from inf to 0.32031, saving model to best_model.h5


656/656 ━━━━━━━━━━━━━━━━━━━━ 81s 121ms/step - MAE: 0.3383 - loss: 0.1541 - val_MAE: 0.3203 - val_loss: 0.1290
Epoch 2/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - MAE: 0.3245 - loss: 0.1328
Epoch 2: val_MAE did not improve from 0.32031
Couches dé-gelées à l'époque 2
656/656 ━━━━━━━━━━━━━━━━━━━━ 75s 115ms/step - MAE: 0.3245 - loss: 0.1328 - val_MAE: 0.3225 - val_loss: 0.1287
Epoch 3/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - MAE: 0.3223 - loss: 0.1313
Epoch 3: val_MAE improved from 0.32031 to 0.31891, saving model to best_model.h5


656/656 ━━━━━━━━━━━━━━━━━━━━ 78s 118ms/step - MAE: 0.3223 - loss: 0.1313 - val_MAE: 0.3189 - val_loss: 0.1278
Epoch 4/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - MAE: 0.3229 - loss: 0.1326
Epoch 4: val_MAE improved from 0.31891 to 0.31717, saving model to best_model.h5


656/656 ━━━━━━━━━━━━━━━━━━━━ 73s 112ms/step - MAE: 0.3229 - loss: 0.1326 - val_MAE: 0.3172 - val_loss: 0.1277
Epoch 5/15
655/656 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - MAE: 0.3209 - loss: 0.1308
Epoch 5: val_MAE did not improve from 0.31717
656/656 ━━━━━━━━━━━━━━━━━━━━ 75s 114ms/step - MAE: 0.3210 - loss: 0.1308 - val_MAE: 0.3188 - val_loss: 0.1278
Epoch 6/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - MAE: 0.3224 - loss: 0.1315
Epoch 6: val_MAE improved from 0.31717 to 0.31262, saving model to best_model.h5


656/656 ━━━━━━━━━━━━━━━━━━━━ 70s 107ms/step - MAE: 0.3224 - loss: 0.1315 - val_MAE: 0.3126 - val_loss: 0.1293
Epoch 7/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - MAE: 0.3223 - loss: 0.1329
Epoch 7: val_MAE did not improve from 0.31262
656/656 ━━━━━━━━━━━━━━━━━━━━ 71s 108ms/step - MAE: 0.3223 - loss: 0.1329 - val_MAE: 0.3243 - val_loss: 0.1297
Epoch 8/15
655/656 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - MAE: 0.3227 - loss: 0.1324
Epoch 8: val_MAE did not improve from 0.31262
656/656 ━━━━━━━━━━━━━━━━━━━━ 74s 113ms/step - MAE: 0.3227 - loss: 0.1324 - val_MAE: 0.3143 - val_loss: 0.1282
Epoch 9/15
656/656 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - MAE: 0.3205 - loss: 0.1308
Epoch 9: val_MAE did not improve from 0.31262
656/656 ━━━━━━━━━━━━━━━━━━━━ 75s 115ms/step - MAE: 0.3205 - loss: 0.1308 - val_MAE: 0.3240 - val_loss: 0.1294
Epoch 9: early stopping
Restoring model weights from the end of the best epoch: 6.


In [142]:
result = model.evaluate(test_ds)
print(f"Test MAE: {result[1]:.4f}")

198/198 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - MAE: 0.3211 - loss: 0.1377
Test MAE: 0.3200
